# Column-Family Stores: Cassandra & HBase

Column-family stores (also called wide-column stores) are NoSQL databases designed for **massive scalability** and **high write throughput**. They organize data into rows and column families, providing flexible schemas while maintaining efficient storage and retrieval patterns.

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Row Key** | Unique identifier for each row; primary index for data access |
| **Column Family** | Logical grouping of columns stored together on disk |
| **Column** | Name-value pair with optional timestamp |
| **Super Column** | Column containing nested columns (deprecated in modern systems) |
| **Keyspace** | Top-level namespace (similar to database in RDBMS) |

## Wide-Column Data Model

Unlike traditional relational tables, wide-column stores allow:
- **Sparse data**: Each row can have different columns
- **Dynamic columns**: Add columns without schema changes
- **Denormalized design**: Data duplicated for read efficiency

```
┌─────────────────────────────────────────────────────────────────┐
│                        KEYSPACE: sensors                        │
├─────────────────────────────────────────────────────────────────┤
│                   COLUMN FAMILY: readings                       │
├──────────────┬──────────────────────────────────────────────────┤
│   Row Key    │              Columns                             │
├──────────────┼──────────────────────────────────────────────────┤
│ sensor_001   │ temp:72.5 | humidity:45 | pressure:1013          │
│ sensor_002   │ temp:68.2 | humidity:52                          │
│ sensor_003   │ temp:75.1 | co2:412 | light:850                  │
└──────────────┴──────────────────────────────────────────────────┘
```

Notice how each row can have **different columns** - this flexibility is a core feature.

## Python Simulation: Column-Family Data Structure

In [ ]:
from typing import Dict, Any, Optional, List
from datetime import datetime
from collections import defaultdict
import hashlib
import json


class Column:
    """Represents a single column with name, value, and timestamp."""
    
    def __init__(self, name: str, value: Any, timestamp: Optional[datetime] = None):
        self.name = name
        self.value = value
        self.timestamp = timestamp or datetime.now()
    
    def __repr__(self):
        return f"Column({self.name}={self.value}, ts={self.timestamp.isoformat()})"
    
    def to_dict(self):
        return {
            "name": self.name,
            "value": self.value,
            "timestamp": self.timestamp.isoformat()
        }


class ColumnFamily:
    """A logical grouping of columns stored together."""
    
    def __init__(self, name: str):
        self.name = name
        self.columns: Dict[str, Column] = {}
    
    def put(self, column_name: str, value: Any, timestamp: Optional[datetime] = None):
        """Insert or update a column (last-write-wins based on timestamp)."""
        new_col = Column(column_name, value, timestamp)
        
        if column_name in self.columns:
            existing = self.columns[column_name]
            # Last-write-wins conflict resolution
            if new_col.timestamp >= existing.timestamp:
                self.columns[column_name] = new_col
        else:
            self.columns[column_name] = new_col
    
    def get(self, column_name: str) -> Optional[Column]:
        return self.columns.get(column_name)
    
    def delete(self, column_name: str):
        """Tombstone deletion (mark as deleted)."""
        if column_name in self.columns:
            del self.columns[column_name]
    
    def get_all_columns(self) -> List[Column]:
        return list(self.columns.values())
    
    def __repr__(self):
        cols = ", ".join(f"{c.name}:{c.value}" for c in self.columns.values())
        return f"ColumnFamily({self.name}: [{cols}])"


print("Column-family classes defined successfully!")

In [ ]:
class Row:
    """A row containing multiple column families, identified by a row key."""
    
    def __init__(self, row_key: str):
        self.row_key = row_key
        self.column_families: Dict[str, ColumnFamily] = {}
    
    def get_or_create_cf(self, cf_name: str) -> ColumnFamily:
        if cf_name not in self.column_families:
            self.column_families[cf_name] = ColumnFamily(cf_name)
        return self.column_families[cf_name]
    
    def put(self, cf_name: str, column_name: str, value: Any):
        cf = self.get_or_create_cf(cf_name)
        cf.put(column_name, value)
    
    def get(self, cf_name: str, column_name: str) -> Optional[Any]:
        cf = self.column_families.get(cf_name)
        if cf:
            col = cf.get(column_name)
            return col.value if col else None
        return None
    
    def __repr__(self):
        return f"Row({self.row_key}): {list(self.column_families.keys())}"


class ColumnFamilyStore:
    """
    Simulates a column-family database with partitioning.
    Inspired by Cassandra/HBase architecture.
    """
    
    def __init__(self, name: str, num_partitions: int = 4):
        self.name = name
        self.num_partitions = num_partitions
        # Each partition holds a subset of rows
        self.partitions: List[Dict[str, Row]] = [{} for _ in range(num_partitions)]
        self.write_count = 0
        self.read_count = 0
    
    def _get_partition(self, row_key: str) -> int:
        """Consistent hash partitioning using MD5."""
        hash_val = int(hashlib.md5(row_key.encode()).hexdigest(), 16)
        return hash_val % self.num_partitions
    
    def put(self, row_key: str, cf_name: str, column_name: str, value: Any):
        """Write a column value to the store."""
        partition_id = self._get_partition(row_key)
        partition = self.partitions[partition_id]
        
        if row_key not in partition:
            partition[row_key] = Row(row_key)
        
        partition[row_key].put(cf_name, column_name, value)
        self.write_count += 1
    
    def get(self, row_key: str, cf_name: str, column_name: str) -> Optional[Any]:
        """Read a column value from the store."""
        partition_id = self._get_partition(row_key)
        partition = self.partitions[partition_id]
        self.read_count += 1
        
        row = partition.get(row_key)
        if row:
            return row.get(cf_name, column_name)
        return None
    
    def get_row(self, row_key: str) -> Optional[Row]:
        """Retrieve entire row."""
        partition_id = self._get_partition(row_key)
        return self.partitions[partition_id].get(row_key)
    
    def scan_partition(self, partition_id: int) -> List[Row]:
        """Scan all rows in a partition."""
        return list(self.partitions[partition_id].values())
    
    def get_stats(self) -> Dict:
        """Get storage statistics."""
        total_rows = sum(len(p) for p in self.partitions)
        partition_sizes = [len(p) for p in self.partitions]
        return {
            "total_rows": total_rows,
            "partition_sizes": partition_sizes,
            "writes": self.write_count,
            "reads": self.read_count
        }


print("ColumnFamilyStore class defined!")

## Example: IoT Sensor Data Storage

In [ ]:
# Create a column-family store for IoT sensor data
iot_store = ColumnFamilyStore("iot_sensors", num_partitions=4)

# Simulate sensor readings with composite row keys
# Row key pattern: sensor_id#date (for time-series partitioning)
sensor_data = [
    ("sensor_001#2026-02-02", "metrics", "temperature", 72.5),
    ("sensor_001#2026-02-02", "metrics", "humidity", 45.2),
    ("sensor_001#2026-02-02", "metrics", "pressure", 1013.25),
    ("sensor_001#2026-02-02", "metadata", "location", "Building A"),
    ("sensor_001#2026-02-02", "metadata", "type", "environmental"),
    
    ("sensor_002#2026-02-02", "metrics", "temperature", 68.1),
    ("sensor_002#2026-02-02", "metrics", "co2_level", 415),
    
    ("sensor_003#2026-02-02", "metrics", "light_level", 850),
    ("sensor_003#2026-02-02", "metrics", "motion_detected", True),
]

# Insert data
for row_key, cf_name, col_name, value in sensor_data:
    iot_store.put(row_key, cf_name, col_name, value)

print("Data inserted!")
print(f"Store stats: {iot_store.get_stats()}")

In [ ]:
# Query specific values
temp = iot_store.get("sensor_001#2026-02-02", "metrics", "temperature")
print(f"Sensor 001 Temperature: {temp}°F")

location = iot_store.get("sensor_001#2026-02-02", "metadata", "location")
print(f"Sensor 001 Location: {location}")

# Get entire row
row = iot_store.get_row("sensor_001#2026-02-02")
print(f"\nFull row: {row}")

# Show column families and their contents
if row:
    for cf_name, cf in row.column_families.items():
        print(f"  {cf_name}: {cf}")

## Write-Optimized Design (LSM Tree)

Column-family stores like Cassandra use **Log-Structured Merge Trees (LSM)** for write optimization:

```
┌─────────────────────────────────────────────────────────────────┐
│                      WRITE PATH                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Client Write                                                │
│       │                                                         │
│       ▼                                                         │
│  2. Commit Log (WAL)  ──► Durability guarantee                  │
│       │                                                         │
│       ▼                                                         │
│  3. MemTable (in-memory sorted structure)                       │
│       │                                                         │
│       ▼  (when full)                                            │
│  4. Flush to SSTable (immutable, sorted on disk)                │
│       │                                                         │
│       ▼  (background)                                           │
│  5. Compaction (merge SSTables, remove tombstones)              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why This is Fast for Writes
- **Sequential I/O**: All writes are sequential (append-only log)
- **No random seeks**: Unlike B-trees that require random disk access
- **Batch amortization**: Compaction happens in background

In [ ]:
class MemTable:
    """In-memory sorted structure (simplified Red-Black tree simulation)."""
    
    def __init__(self, max_size: int = 100):
        self.data: Dict[str, Dict[str, Any]] = {}  # Sorted by key
        self.max_size = max_size
        self.size = 0
    
    def put(self, key: str, column: str, value: Any) -> bool:
        """Returns True if MemTable needs flushing."""
        if key not in self.data:
            self.data[key] = {}
        self.data[key][column] = value
        self.size += 1
        return self.size >= self.max_size
    
    def get(self, key: str, column: str) -> Optional[Any]:
        return self.data.get(key, {}).get(column)
    
    def to_sstable(self) -> 'SSTable':
        """Flush to immutable SSTable."""
        sorted_data = dict(sorted(self.data.items()))
        return SSTable(sorted_data)
    
    def clear(self):
        self.data = {}
        self.size = 0


class SSTable:
    """Sorted String Table - immutable, sorted on-disk structure."""
    
    def __init__(self, data: Dict[str, Dict[str, Any]]):
        self.data = data
        self.bloom_filter = self._build_bloom_filter()
        self.index = self._build_sparse_index()
        self.created_at = datetime.now()
    
    def _build_bloom_filter(self) -> set:
        """Simplified bloom filter (actual uses bit array with multiple hashes)."""
        return set(self.data.keys())
    
    def _build_sparse_index(self) -> Dict[str, int]:
        """Sparse index for faster lookups."""
        keys = list(self.data.keys())
        # Index every 10th key
        return {keys[i]: i for i in range(0, len(keys), 10)} if keys else {}
    
    def might_contain(self, key: str) -> bool:
        """Bloom filter check - false positives possible, no false negatives."""
        return key in self.bloom_filter
    
    def get(self, key: str, column: str) -> Optional[Any]:
        if not self.might_contain(key):
            return None
        return self.data.get(key, {}).get(column)


print("LSM-Tree components defined!")

In [ ]:
class LSMStore:
    """Complete LSM-based column-family store."""
    
    def __init__(self, memtable_size: int = 10):
        self.memtable = MemTable(memtable_size)
        self.sstables: List[SSTable] = []  # Oldest to newest
        self.commit_log: List[tuple] = []  # WAL simulation
    
    def put(self, key: str, column: str, value: Any):
        # Step 1: Write to commit log (WAL)
        self.commit_log.append((key, column, value, datetime.now()))
        
        # Step 2: Write to MemTable
        needs_flush = self.memtable.put(key, column, value)
        
        # Step 3: Flush if MemTable is full
        if needs_flush:
            self._flush_memtable()
    
    def _flush_memtable(self):
        """Flush MemTable to SSTable."""
        if self.memtable.size > 0:
            sstable = self.memtable.to_sstable()
            self.sstables.append(sstable)
            self.memtable.clear()
            print(f"Flushed MemTable to SSTable #{len(self.sstables)}")
    
    def get(self, key: str, column: str) -> Optional[Any]:
        # Check MemTable first (most recent)
        value = self.memtable.get(key, column)
        if value is not None:
            return value
        
        # Check SSTables from newest to oldest
        for sstable in reversed(self.sstables):
            value = sstable.get(key, column)
            if value is not None:
                return value
        
        return None
    
    def compact(self):
        """Merge all SSTables into one (simplified compaction)."""
        if len(self.sstables) < 2:
            return
        
        merged_data = {}
        for sstable in self.sstables:
            for key, columns in sstable.data.items():
                if key not in merged_data:
                    merged_data[key] = {}
                merged_data[key].update(columns)
        
        self.sstables = [SSTable(dict(sorted(merged_data.items())))]
        print(f"Compacted to 1 SSTable with {len(merged_data)} rows")


# Demonstrate LSM writes
lsm = LSMStore(memtable_size=5)

# Insert data that will trigger flushes
for i in range(12):
    lsm.put(f"row_{i:03d}", "value", f"data_{i}")

print(f"\nTotal SSTables: {len(lsm.sstables)}")
print(f"MemTable size: {lsm.memtable.size}")

## Partitioning Strategies

Effective partitioning is crucial for scalability and performance.

### 1. Hash Partitioning (Cassandra Default)
- Partition key hashed to determine node
- Even data distribution
- No range queries across partitions

### 2. Range Partitioning (HBase)
- Data sorted by row key
- Enables efficient range scans
- Risk of hotspots

### 3. Composite Partitioning
- Combine partition key + clustering key
- Group related data on same partition

In [ ]:
class PartitioningDemo:
    """Demonstrate different partitioning strategies."""
    
    @staticmethod
    def hash_partition(key: str, num_nodes: int) -> int:
        """Murmur3-like hash partitioning (Cassandra style)."""
        hash_val = int(hashlib.md5(key.encode()).hexdigest(), 16)
        return hash_val % num_nodes
    
    @staticmethod
    def range_partition(key: str, ranges: List[str]) -> int:
        """Range-based partitioning (HBase style)."""
        for i, boundary in enumerate(ranges):
            if key < boundary:
                return i
        return len(ranges)
    
    @staticmethod
    def composite_key(partition_key: str, clustering_key: str) -> str:
        """Create composite key for time-series data."""
        return f"{partition_key}#{clustering_key}"


# Demonstrate partitioning
demo = PartitioningDemo()

# Hash partitioning example
print("Hash Partitioning (4 nodes):")
sensors = ["sensor_001", "sensor_002", "sensor_003", "sensor_004", "sensor_005"]
for sensor in sensors:
    node = demo.hash_partition(sensor, 4)
    print(f"  {sensor} → Node {node}")

# Range partitioning example
print("\nRange Partitioning (alphabetical boundaries):")
ranges = ["h", "p", "x"]  # Boundaries: <h, h-p, p-x, >x
keys = ["apple", "mango", "zebra", "grape"]
for key in keys:
    partition = demo.range_partition(key, ranges)
    print(f"  {key} → Partition {partition}")

In [ ]:
# Time-series partitioning strategy
print("Time-Series Composite Key Strategy:")
print("="*50)

# Pattern: sensor_id#date as partition key, timestamp as clustering key
# This groups one day's data for a sensor on the same partition

time_series_keys = [
    ("sensor_001", "2026-02-02", "10:00:00"),
    ("sensor_001", "2026-02-02", "10:05:00"),
    ("sensor_001", "2026-02-02", "10:10:00"),
    ("sensor_001", "2026-02-03", "10:00:00"),  # New partition
    ("sensor_002", "2026-02-02", "10:00:00"),  # Different sensor
]

partition_map = defaultdict(list)

for sensor, date, time in time_series_keys:
    partition_key = f"{sensor}#{date}"
    clustering_key = time
    partition_id = demo.hash_partition(partition_key, 4)
    partition_map[partition_key].append(clustering_key)
    print(f"  {partition_key}:{clustering_key} → Node {partition_id}")

print("\nData grouped by partition key:")
for pk, times in partition_map.items():
    print(f"  {pk}: {times}")

## When to Use Wide-Column Stores

### ✅ Ideal Use Cases

| Use Case | Why Column-Family Works |
|----------|-------------------------|
| **Time-Series Data** | Sequential writes, range queries by time |
| **IoT Sensor Data** | High write throughput, sparse columns |
| **Event Logging** | Append-only, chronological access |
| **User Activity Feeds** | Wide rows, sorted by timestamp |
| **Metrics/Analytics** | Aggregation-friendly, columnar access |
| **Messaging Systems** | Message queues with time-based partitioning |

### ❌ Poor Fit Scenarios

| Scenario | Why Not Column-Family |
|----------|------------------------|
| Complex JOINs | No native join support |
| ACID transactions | Limited transaction scope |
| Ad-hoc queries | Requires predefined access patterns |
| Small datasets | Overhead not justified |
| Frequent updates | Not optimized for in-place updates |

In [ ]:
# Complete example: Time-series IoT data model

class IoTTimeSeriesStore:
    """Production-like IoT data store with Cassandra-style modeling."""
    
    def __init__(self):
        self.store = ColumnFamilyStore("iot_timeseries", num_partitions=8)
    
    def _make_partition_key(self, device_id: str, date: str) -> str:
        """Partition by device + date for bounded partition size."""
        return f"{device_id}#{date}"
    
    def record_reading(self, device_id: str, timestamp: datetime, 
                       readings: Dict[str, Any]):
        """Record sensor readings."""
        date_str = timestamp.strftime("%Y-%m-%d")
        time_str = timestamp.strftime("%H:%M:%S.%f")
        row_key = self._make_partition_key(device_id, date_str)
        
        for metric, value in readings.items():
            # Column name includes timestamp for clustering
            col_name = f"{time_str}:{metric}"
            self.store.put(row_key, "readings", col_name, value)
    
    def get_daily_readings(self, device_id: str, date: str) -> List[Dict]:
        """Get all readings for a device on a specific date."""
        row_key = self._make_partition_key(device_id, date)
        row = self.store.get_row(row_key)
        
        if not row or "readings" not in row.column_families:
            return []
        
        results = []
        cf = row.column_families["readings"]
        for col in sorted(cf.columns.values(), key=lambda c: c.name):
            time_str, metric = col.name.rsplit(":", 1)
            results.append({
                "time": time_str,
                "metric": metric,
                "value": col.value
            })
        return results


# Demo
ts_store = IoTTimeSeriesStore()

# Record sensor data
from datetime import timedelta

base_time = datetime(2026, 2, 2, 10, 0, 0)
for i in range(5):
    ts = base_time + timedelta(minutes=i*5)
    ts_store.record_reading(
        device_id="temp_sensor_001",
        timestamp=ts,
        readings={
            "temperature": 70 + i * 0.5,
            "humidity": 45 + i * 0.2
        }
    )

# Query daily readings
readings = ts_store.get_daily_readings("temp_sensor_001", "2026-02-02")
print("Daily readings for temp_sensor_001:")
for r in readings:
    print(f"  {r['time']} - {r['metric']}: {r['value']}")

## Cassandra vs HBase Comparison

| Feature | Cassandra | HBase |
|---------|-----------|-------|
| **Architecture** | Peer-to-peer, no master | Master-slave (HMaster) |
| **Consistency** | Tunable (ONE, QUORUM, ALL) | Strong consistency |
| **Query Language** | CQL (SQL-like) | Java API, Thrift |
| **Partitioning** | Consistent hashing | Range-based |
| **Ecosystem** | Standalone | Hadoop ecosystem |
| **Write Speed** | Excellent | Very good |
| **Range Scans** | Limited | Excellent |
| **Operations** | Simpler | More complex |

## 📌 Key Takeaways

### Data Model
- **Wide-column** = rows with dynamic, sparse columns grouped into column families
- **Row key** is the primary index; design keys for your access patterns
- **Denormalization** is expected; duplicate data for read efficiency

### Write Optimization
- **LSM Trees** enable O(1) writes via sequential I/O
- **MemTable → SSTable → Compaction** pipeline
- Trade read complexity for write speed

### Partitioning
- **Hash partitioning** for even distribution (Cassandra)
- **Range partitioning** for sequential scans (HBase)
- **Composite keys** for time-series: `partition_key#clustering_key`

### Best Practices
1. **Model for queries**, not relationships
2. **Bound partition sizes** (< 100MB recommended)
3. **Use time-bucketing** for time-series data
4. **Avoid hotspots** with proper key design
5. **Accept eventual consistency** for scalability

### When to Choose
- ✅ High write throughput, time-series, IoT, logging
- ❌ Complex transactions, ad-hoc queries, small datasets